In [1]:
import sys
sys.path.append('../')
import torch
import torch.nn as nn
import torch.optim as optim
from Code import NewNeurons as Neurons
#from Code import macroNeurons as Neurons
from Code.lstm import lstmPolicyPredictor, FullyConnected
from Code.envs.MountainCar import MultiMountainCar, LookupPolicy, PassiveEnv
from Code.SNN import RSNN, FeedForwardSNN, magicRSNN, AdaptiveFF, newRSNN
from Code.DynamicNetwork import DynNetwork, SequenceWrapper
import time
from collections import OrderedDict

In [2]:
#TODO: output reset mechanism

config = {
    'ALPHA': 0.7,
    'BETA': 0.95,
    'RESET_ZERO': False,
    'THRESH_ADD': 1,
    'THRESH_DECAY': 1,
    'DECODING': 'potential',
    'SPIKE_FN': 'ss'
}

outconfig = {
    'ALPHA': 0,
    'BETA': 0,
    'RESET_ZERO': False,
    'THRESH_ADD': 1,
    'THRESH_DECAY': 1,
    'DECODING': 'potential',
    'SPIKE_FN': 'ss'
}


architecture = OrderedDict([
    ('input', [1]),
    ('pre_mem', [64, ['input', 'mem'], Neurons.LIFNeuron, config]),
    ('mem', [32, ['pre_mem'], Neurons.CooldownNeuron, config]), #CooldownNeuron
    ('post_mem', [64, ['input', 'mem'], Neurons.LIFNeuron, config]),
    ('output', [1, ['post_mem'], Neurons.OutputNeuron, outconfig]), #OutputNeuron
])

device = torch.device('cuda')

#Neurons.set_config(config)

In [3]:
torch.backends.cudnn.benchmark

False

In [4]:
env = PassiveEnv(device)

In [5]:
BATCH_SIZE = 1024
SIM_TIME = 1

In [6]:
#model = lstmPolicyPredictor(2,8,16)
#model = RSNN(config, 1, 32, 32, 1, Neurons.LIFNeuron, Neurons.AdaptiveNeuron, Neurons.OutputNeuron)
#model = magicRSNN(config, 1, 32, 32, 1, Neurons.LIFNeuron, Neurons.MagicNeuron, Neurons.OutputNeuron)
#model = FeedForwardSNN(config, [1, 128, 128, 1], Neurons.LIFNeuron, Neurons.OutputNeuron)
#model = FullyConnected([1, 128, 128, 1])
#model = AdaptiveFF(config, 1, 64, 32, 64, 1, Neurons.LIFNeuron, Neurons.AdaptiveNeuron, Neurons.OutputNeuron)
#model = AdaptiveFF(config, 1, 64, 32, 64, 1, Neurons.LIFNeuron, Neurons.FlipFlopNeuron, Neurons.OutputNeuron)
#model = RSNN(config, 1, 32, 32, 1, Neurons.LIFNeuron, Neurons.FlipFlopNeuron, Neurons.OutputNeuron)
#model = newRSNN(config, 1, 32, 32, 32, 1, Neurons.LIFNeuron, Neurons.FlipFlopNeuron, Neurons.OutputNeuron)
#model = newRSNN(config, 1, 64, 32, 64, 128, 1, Neurons.CooldownNeuron, Neurons.CooldownNeuron, Neurons.OutputNeuron)

#model = newRSNN(config, 1, 64, 32, 64, 128, 1, Neurons.LIFNeuron, Neurons.CooldownNeuron, Neurons.OutputNeuron)
model = DynNetwork(architecture, SIM_TIME)

teacher = LookupPolicy(device)


#TODO: test superspike instead of Bellec



In [7]:

model = SequenceWrapper(model, BATCH_SIZE, device, True)

/opt/conda/lib/python3.6/site-packages/torch/jit/__init__.py:858: UserWarning: `optimize` is deprecated and has no effect. Use `with torch.jit.optimized_execution() instead
  warnings.warn("`optimize` is deprecated and has no effect. Use `with torch.jit.optimized_execution() instead")


In [8]:
spikes = torch.ones((13, 1, config['SIM_TIME']))
def logger(h, t, i):
    if t%10 == 0:
        spikes[t//10, 0, i] = h['spikes'][0][:32].sum()
        #spikes[t//10, 1, i] = h['spikes'][0][32:].sum()

        #print(t, i)
        #print(h['spikes'][0].sum())
    

KeyError: 'SIM_TIME'

In [ ]:
#torch.autograd.set_detect_anomaly(True)

In [9]:
#bce = nn.BCELoss(reduction='none') #reduction='sum'
mse = nn.MSELoss(reduction='none')
optimizer = optim.Adam(model.parameters(), lr=1e-4)#0.000011e-6

In [10]:
start = time.time()
#inputs, targets, mask = env.getBatch(BATCH_SIZE)
#model(inputs/0.4, h=None, logger=logger)
#print(spikes.squeeze())
for i in range(5000):
    model.zero_grad()
    inputs, targets, mask = env.getBatch(BATCH_SIZE)
    if i%100 == 0:
        # torch.autograd.set_detect_anomaly(False)
        for p in model.parameters():
            if torch.isnan(p).any():
                raise Exception('Corrupted Model')
    outputs, _ = model(inputs/0.4)
    #print(outputs.shape, targets.shape)
    loss = (mse(outputs.squeeze(dim=2)/50, targets) * mask).sum() / mask.sum()
    loss.backward()
    optimizer.step()
    #if i%100 == 0:
    #    torch.autograd.set_detect_anomaly(False)
    if i%10 == 0:
        print(loss.item(), (loss/targets.view(-1).var()).item(), i) #, ((outputs>0.5) != targets).sum()
    #if i%50 == 0:
    #    model(inputs/0.4, h=None, logger=logger)
    #    print(spikes.squeeze())
print(time.time()-start)

0.0007838220335543156 1.4163185358047485 0
0.0007259412668645382 1.3354312181472778 10
0.0006706736166961491 1.2438833713531494 20
0.0006284828996285796 1.1412580013275146 30
0.0005815805634483695 1.0962244272232056 40
0.0005576988332904875 1.0025904178619385 50
0.0005259020836092532 0.9616990685462952 60
0.0004957220517098904 0.910566508769989 70
0.00047945367987267673 0.8804805278778076 80
0.0004688745830208063 0.8716951608657837 90
0.000453483167802915 0.8263423442840576 100
0.0004429788968991488 0.8052365183830261 110
0.0004171623440925032 0.7625855207443237 120
0.0004027513787150383 0.7361750602722168 130
0.0003775546501856297 0.6999691724777222 140
0.00036375439958646894 0.6727019548416138 150
0.00033988826908171177 0.6269350051879883 160
0.0003146407543681562 0.566472589969635 170
0.0002926280430983752 0.5298851728439331 180
0.00028517161263152957 0.5217688083648682 190
0.00027417144156061113 0.5003039240837097 200
0.0002516015956643969 0.46277615427970886 210
0.0002459955576341

In [ ]:
model.adaptive_linear.weight

In [ ]:
#torch.save(model, '../models/lstm_passive')
#0.0002

In [ ]:
#torch.save(model, '../models/withalpha_006')
#0.001

In [ ]:
from matplotlib import pyplot as plt
#model = torch.load('../models/snn_passive3')
%matplotlib



In [ ]:
def doplot():
    inputs, targets, mask = env.getBatch(1)
    outputs, _ = model(inputs/0.4)
    plt.close()
    plt.plot(inputs[:, 0, 0], targets)
    plt.plot(inputs[:, 0, 0], outputs.squeeze().detach()/50)

In [ ]:
doplot()

In [ ]:

for p in model.parameters():
    print(p.shape)

In [ ]:
for p in model.parameters():
    print(p.shape)

In [ ]:
model.parameters().__next__()

In [ ]:
#model(inputs/0.4)[0].squeeze()
model.zero_grad()
inputs, targets, mask = env.getBatch(1)
outputs = model(inputs/0.4)
loss = (mse(outputs.squeeze(dim=2)/30, targets) * mask).sum() / mask.sum()
loss.backward()

In [ ]:
teacher(torch.cat((torch.zeros([20, 1]), torch.linspace(-0.07, 0, 20).unsqueeze(1)), dim=1))

In [ ]:
torch.cat((torch.zeros([20, 1]), torch.linspace(-0.07, 0, 20).unsqueeze(1)), dim=1)

In [ ]:
teacher(torch.tensor([0, -0.052]))

In [ ]:

model

In [ ]:

print(model.model.graph)

In [ ]:

torch.set_printoptions(profile='default')

In [ ]:
model.parameters().__next__()[0,0] 

In [ ]:
pretrace.parameters().__next__()[0,0]= 0.321


In [ ]:
print(model.model.layers.mem.code)

In [ ]:
import torch.jit

In [ ]:
torch.jit.optimized_execution()

In [ ]:
torch._C._get_graph_executor_optimize()

In [11]:
torch.get_num_interop_threads()

4

In [ ]:
torch.jit._fork
